In [13]:
import pandas as pd
import urllib.parse
import os

In [14]:
csv_path = 'loadtest_stats_history.csv'
parquet_dir_path = 'loadtest_stats_history'

if not os.path.isdir(parquet_dir_path): os.mkdir(parquet_dir_path)

def df_to_parquet(df, target_dir, chunk_size=1000000, **parquet_wargs):
    """Writes pandas DataFrame to parquet format with pyarrow.

    Args:
        df: DataFrame
        target_dir: local directory where parquet files are written to
        chunk_size: number of rows stored in one chunk of parquet file. Defaults to 1000000.
    """    
    for i in range(0, len(df), chunk_size):
        print(i)
        slc = df.iloc[i : i + chunk_size]
        chunk = int(i/chunk_size)
        fname = os.path.join(target_dir, f"part_{chunk:04d}.parquet.gzip")
        slc.to_parquet(fname, engine="pyarrow", compression='gzip', **parquet_wargs)

In [15]:
df = pd.read_csv(csv_path)

In [16]:
df['is_agg'] = df['Name'].str.startswith('Aggregated')
df['is_user_get'] = df['Name'].str.startswith('/user/get/')
df['is_user_search'] = df['Name'].str.startswith('/user/search')

In [17]:
df['op_type'] = df.apply(lambda x: 'agg' if x['is_agg'] else 'get' if x['is_user_get'] else 'search' if x['is_user_search'] else None, axis=1)

In [18]:
df['op_type'].value_counts(dropna=False)

get       4607705
search    4303756
agg           414
Name: op_type, dtype: int64

In [19]:
df.loc[df.is_user_get, 'get_id'] = df.loc[df.is_user_get, 'Name'].str.slice(10)

In [20]:
df.loc[df.is_user_search, 'search_fname'] = df.loc[df.is_user_search, 'Name'].str.slice(13).str.split('&').apply(lambda x: x[0])
df.loc[df.is_user_search, 'search_fname'] = df.loc[df.is_user_search, 'search_fname'].str.split('=').apply(lambda x: urllib.parse.unquote(x[1]))

In [21]:
df.loc[df.is_user_search, 'search_lname'] = df.loc[df.is_user_search, 'Name'].str.slice(13).str.split('&').apply(lambda x: x[1])
df.loc[df.is_user_search, 'search_lname'] = df.loc[df.is_user_search, 'search_lname'].str.split('=').apply(lambda x: urllib.parse.unquote(x[1]))

In [22]:
df.drop(columns=['Name'], inplace=True)

In [23]:
df.groupby(['User Count', 'op_type'], dropna=False)[['Timestamp']].count()

Timestamp
User Count op_type           
0          agg             14
           get         215313
           search      187252
1          agg            159
           get         299570
           search      292202
10         agg            107
           get         985938
           search      962974
100        agg             73
           get        1445429
           search     1350718
976        agg              1
           get          30759
           search       27647
1000       agg             60
           get        1630696
           search     1482963

In [37]:
# df

In [35]:
df_to_parquet(df, parquet_dir_path)

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
